# Introduction and Background

In this notebook, we demonstrate how to adapt the kernel methods shown in the [previous notebook](3_KernelMethods.ipynb) to use sparse kernels.

As for the previous notebooks, for each model, we first go step-by-step through the derivation, with equations, embedded links, and citations supplied where useful. At the end of the notebook, we employ a "skcosmo class" for the model, which is found in the skcosmo module and contains all necessary functions.

In [ ]:
#!/usr/bin/env python3
import sys

# Maths things
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Local Utilities for Notebook
sys.path.append('../')
from utilities.general import load_variables, get_stats
from utilities.plotting import (
    plot_projection, plot_regression, check_mirrors, get_cmaps, table_from_dict
)
from sklearn.metrics.pairwise import linear_kernel, rbf_kernel
from sklearn.kernel_ridge import KernelRidge
from sklearn.decomposition import KernelPCA
from skcosmo.decomposition import KernelPCovR
from skcosmo.sample_selection import FPS
from skcosmo.preprocessing import SparseKernelCenterer
from functools import partial

cmaps = get_cmaps()
plt.style.use('../utilities/kernel_pcovr.mplstyle')
dbl_fig=(2*plt.rcParams['figure.figsize'][0], plt.rcParams['figure.figsize'][1])

First, we must load the data. For a step-by-step explanation of this, please see [Importing Data](X_ImportingData.ipynb).

In [ ]:
var_dict = load_variables()
locals().update(var_dict)

# Constructing a Sparse Kernel

In [ ]:
## Change this cell to change the kernel function throughout
kernel_params = {"kernel": "rbf", "gamma": 1.0}
kernel_func = partial(rbf_kernel, gamma=1.0)
kernel_type = "gaussian"

## The Nystr&ouml;m Approximation

In sparse kernel methods, an approximate kernel is used in place of the full kernel. This approximate kernel is typically constructed according to the [Nystr&ouml;m approximation](https://en.wikipedia.org/wiki/Low-rank_matrix_approximations#Nystr%C3%B6m_approximation) [(Williams 2001)](http://papers.nips.cc/paper/1866-using-the-nystrom-method-to-speed-up-kernel-machines.pdf),

\begin{equation}
    \mathbf{K} \approx \mathbf{\hat{K}}_{NN} = \mathbf{K}_{NM} \mathbf{K}_{MM}^{-1} \mathbf{K}_{NM}^T
\end{equation}

Here, $M$ represents a subset of the $N$ total rows/columns of the kernel matrix, i.e. the kernel between a small **active set** that is selected with subsampling method, like farthest point sampling (FPS) [(Eldar 1997)](https://doi.org/10.1109/83.623193), or a CUR decomposition [(Imbalzano2018)](https://doi.org/10.1063/1.5024611), that is discussed in the [next notebook](5_CUR.ipynb).

**Note**: In our imported data from `load_variables()`, `X_train` and `X_test` are pre-centered and pre-scaled relative to the train set. Additionally, the imported `K_train` and `K_test` kernels have been constructed using uncentered and unscaled $\mathbf{X}$ data. If we want to compare the sparse kernels that we will soon construct to the imported "full" kernels, we also need to build the sparse kernels on uncentered and unscaled $\mathbf{X}$ data. Therefore, we undo the scaling and centering on the imported $\mathbf{X}$ data here, and re-center and re-scale the data after building the sparse kernels. In general, centering and scaling the $\mathbf{X}$ data before building kernels is optional; however, one should be consistent when working with multiple kernels.

In [ ]:
X_train = X_train * X_scale + X_center
X_test = X_test * X_scale + X_center

In [ ]:
n_active = 20

fps_selector = FPS(n_to_select=n_active)
fps_idxs = fps_selector.fit(X_train).selected_idx_

Xsparse = X_train[fps_idxs, :]

In other words, $\mathbf{K}_{NM}$ is the kernel matrix between input data $\mathbf{X}$ and $\mathbf{X_{sparse}}$, a version of $\mathbf{X}$ containing only the active set. $\mathbf{K}_{MM}$ is the matrix containing the kernel evaluated between the active set samples.

In [ ]:
Kmm = kernel_func(Xsparse, Xsparse)
Knm_train = kernel_func(X_train, Xsparse)
Knm_test = kernel_func(X_test, Xsparse)

## The Explicit RKHS

Sometimes, it might be more convenient to explicitly write out the projection of the training points
on the [RKHS](https://en.wikipedia.org/wiki/Reproducing_kernel_Hilbert_space) defined by the active set.
This is essentially a KPCA built for the active set, that is not truncated to a few eigenvectors,


\begin{equation}
    \mathbf{\Phi}_{NM} = \mathbf{K}_{NM} \mathbf{U}_{MM} \mathbf{\Lambda}_{MM}^{-1/2}.
\end{equation}

Using this definition it is easy to derive the Nyström approximation: 

\begin{equation}
\mathbf{\hat{K}}_{NN} = \mathbf{\Phi}_{NM} \mathbf{\Phi}_{NM}^T = 
\mathbf{K}_{NM}  \mathbf{U}_{MM} \mathbf{\Lambda}_{MM}^{-1}  \mathbf{U}_{MM}^T \mathbf{K}_{NM}^T
= \mathbf{K}_{NM} \mathbf{K}_{MM}^{-1} \mathbf{K}_{NM}^T.
\end{equation}

## Centering the RKHS

Just as the "full" kernels in [the previous notebook](3_KernelMethods.ipynb) were centered, sparse kernels must also be centered relative to the train set. The goal is to ensure that the Nyström-approximated kernel $\mathbf{\hat{K}}_{NN}$ is centered relative to the training set. This is achieved by centering the approximate RKHS features $\mathbf{\Phi}_{NM}$, and we denote the centered version of the RKHS features as $\mathbf{\tilde{\Phi}}_{NM} = \mathbf{\Phi}_{NM} - \mathbf{\bar{\Phi}}_{M}$. If we represent each element of $\mathbf{\Phi}$ in its summation form

\begin{equation}
    \mathbf{\Phi}_{nm} = \frac{1}{\sqrt{\Lambda_{mm}}}\sum_{m'}^M \left(K_{nm'} U_{m'm}\right), 
\end{equation}

then the column means are given by

\begin{equation}
    \mathbf{\bar{\Phi}}_{m} = \frac{1}{\sqrt{\Lambda_{mm}}}\sum_{m'}^M \left(\left(\frac{1}{N}\sum_n^N K_{nm'}\right)U_{m'm} \right), 
\end{equation}

so the centered feature matrix is computed by $\mathbf{K}_{NM}$, centered by the column means of the kernel, as denoted by $\mathbf{\bar{K}}_M$.

\begin{equation}
    \mathbf{\tilde{\Phi}}_{NM} =  \left(\mathbf{K}_{NM} -\mathbf{\bar{K}}_M\right) \mathbf{U}_{MM} \mathbf{\Lambda}_{MM}^{-1/2}.
\end{equation}

It is best to keep the column mean $\mathbf{\bar{K}}_M$ separate, because it has to be used also when performing out-of-sample embedding, where $\mathbf{K}_{NM}$ would corresponds to the test set kernel. For consistency, $\mathbf{\bar{K}}_M$ must always be the kernel mean associated with the **train set**.

Alternatively, one can store $\mathbf{\bar{\Phi}}_{M}$ and use it for centering.

**Note**: in the following we often use $\mathbf{\Phi}_{NM}$ and $\mathbf{\tilde{\Phi}}_{NM}$ without the subscripts to indicate the train set features approximated in the active RKHS.

<!---
\begin{equation}
    \mathbf{\tilde{\Phi}}_{nm} = \frac{1}{\sqrt{\Lambda_{mm}}}\sum_{m'}^M \left(\left(K_{nm'} - \frac{1}{N}\sum_{n'}^N K_{n'm'}\right)U_{m'm}\right)
\end{equation}
\end{comment}
--->

The kernel between the active points $\mathbf{K}_{MM}$ can also be centered independently, though this is optional and can lead to near-zero eigenvalues, as noted below.

In [ ]:
K_center = np.mean(Knm_train, axis=0)

Knm_train -= K_center
Knm_test -= K_center

In addition to centering the kernel, one may also want to scale the sparse kernel(s) so that, for example, the trace of the Nyström-approximated train kernel is equal to the number of training points. To achieve this, the sparse kernel $\mathbf{K}_{NM}$ is divided by

\begin{equation}
    \sqrt{\frac{\operatorname{Tr}(\mathbf{K}_{NM}\mathbf{K}_{MM}^{-1}\mathbf{K}_{NM}^T)}{n_{train}}}
\end{equation}

where $\mathbf{K}_{NM}$ refers to the kernel between the **train set** and the active points. The same scaling should be applied to both the train and test sets.

In [ ]:
K_scale = Knm_train @ np.linalg.pinv(Kmm, rcond=1.0E-12) @ Knm_train.T
K_scale = np.sqrt(np.trace(K_scale) / Knm_train.shape[0])

Knm_train /= K_scale
Knm_test /= K_scale

In computing the RKHS features, it might be wise to discard some of the smaller eigenvalues. For instance, if it has been centered, $\mathbf{K}_{MM}$ has one _exactly_ zero eigenvalue, and we should take it out of the projection. [(Honeine 2014)](https://arxiv.org/pdf/1407.2904.pdf)

In [ ]:
vmm, Umm = np.linalg.eigh(Kmm)

# Umm/vmm are already sorted, but in *increasing* order, so reverse them
Umm = np.flip(Umm, axis=1)
vmm = np.flip(vmm, axis=0)

Umm = Umm[:, vmm > 0]
vmm = vmm[vmm > 0]

Phi = (
    Knm_train
    @ Umm[:, : n_active - 1]
    @ np.diagflat(1.0 / np.sqrt(vmm[0 : n_active - 1]))
)

In [ ]:
# Re-center and scale the X data
X_train = (X_train - X_center) / X_scale
X_test = (X_test - X_center) / X_scale

# Sparse KPCA

Sparse kernel principal component analysis (sKPCA) is formulated in the same way as standard KPCA, with the exception that an approximate kernel matrix is used. 

$\mathbf{\tilde{\Phi}}$ is the feature matrix for the train points in the RKHS defined by the $M$ active set. Sparse KPCA can be understood (and derived) as PCA in the active set RKHS, by computing and diagonalising the covariance matrix built from $\mathbf{\tilde{\Phi}}$. The covariance should be computed using *centered* kernel features, as discussed above

\begin{equation}
\mathbf{C} = \mathbf{\tilde{\Phi}}^T \mathbf{\tilde{\Phi}} = \mathbf{U}_C \mathbf{\Lambda}_C \mathbf{U}_C^T.
\end{equation}

In [ ]:
C = np.dot(Phi.T, Phi)

v_C, U_C = np.linalg.eigh(C)

# U_C/v_C are already sorted, but in *increasing* order, so reverse them
U_C = np.flip(U_C, axis=1)
v_C = np.flip(v_C, axis=0)

U_C = U_C[:, v_C > 0]
v_C = v_C[v_C > 0]

## Projecting the Sparse KPCA
Projecting into latent space, we get

\begin{align}
    \mathbf{T} &= \mathbf{\tilde{\Phi}} \hat{\mathbf{U}}_C \\
    &= \left(\mathbf{K}_{NM}- \bar{\mathbf{K}}_M\right)\mathbf{U}_{MM} \mathbf{\Lambda}_{MM}^{-1/2} \hat{\mathbf{U}}_C \\
    &= \mathbf{K}_{NM}\mathbf{U}_{MM} \mathbf{\Lambda}_{MM}^{-1/2} \hat{\mathbf{U}}_C -\bar{\mathbf{\Phi}}\hat{\mathbf{U}}_C\\
    &= \mathbf{K}_{NM} \mathbf{P}_{KT} - \mathbf{\bar{T}}
\end{align}

where our sKPCA projector from kernel space $\mathbf{P}_{KT} = \mathbf{U}_{MM} \mathbf{\Lambda}_{MM}^{-1/2}\mathbf{\hat{U}}_C$, where $\mathbf{\hat{U}}_C$ contains the first $n_{PCA}$ eigenvectors of $\mathbf{C}$. $\mathbf{\bar{T}} = \bar{\mathbf{\Phi}}\hat{\mathbf{U}}_C$ centers in the latent space, and is computed and stored for future use.

In [ ]:
PKT = (
    Umm[:, : n_active - 1]
    @ np.diagflat(1.0 / np.sqrt(vmm[0 : n_active - 1]))
    @ U_C[:, :n_PC]
)

T_train = Knm_train @ PKT
T_test = Knm_test @ PKT

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=dbl_fig)

ref_kpca = KernelPCA(
    n_components=n_PC, kernel="rbf", gamma=1.0, fit_inverse_transform=True
)
ref_kpca.fit(X_train)
T_kpca = ref_kpca.transform(X_test)

plot_projection(
    Y_test,
    check_mirrors(T_test, T_kpca),
    fig=fig,
    ax=axes[0],
    title="Sparse KPCA on {} Environments".format(Kmm.shape[0]),
    **cmaps
)
plot_projection(
    Y_test,
    T_kpca,
    fig=fig,
    ax=axes[1],
    title="KPCA on {} Environments".format(X_train.shape[0]),
    **cmaps
)

fig.subplots_adjust(wspace=0.4)

We can also reconstruct $\mathbf{X}$ using $\mathbf{P}_{TX} = \mathbf{\Lambda}^{-1}\mathbf{T}^T\mathbf{X}$, as in KPCA.

In [ ]:
PTX = np.diagflat(1.0 / (v_C[:n_PC])) @ T_train.T @ X_train

Xr_test = T_test @ PTX

## Error and Loss

The same loss functions are used as in KPCA, so we can compare the loss with that of KPCA

In [ ]:
K_approx_train = T_train @ T_train.T

K_test_test = kernel_func(X_test, X_test)
K_approx_test = T_test @ T_test.T

table_from_dict(
    [
        get_stats(
            x=X_test,
            xr=ref_kpca.inverse_transform(T_kpca),
            y=Y_test,
            t=T_kpca,
            k=K_test,
            kapprox=T_kpca @ T_kpca.T,
        ),
        get_stats(
            x=X_test,
            xr=Xr_test,
            y=Y_test,
            t=T_test,
            k=K_test_test,
            kapprox=K_approx_test,
        ),
    ],
    headers=["KPCA", "sKPCA"],
    title="sKPCA",
)

# Sparse KRR

## Sparse KRR Weights
Let's see how sparsity works out in the case of regression. 

If we now build a (regularized) linear regression in the RKHS we get the loss

\begin{equation}
\ell = \lVert \mathbf{Y} - \mathbf{\tilde{\Phi}}\mathbf{P}_{\mathbf{\tilde{\Phi}} Y} \rVert^2 + 
\lambda \lVert\mathbf{P}_{\mathbf{\tilde{\Phi}} Y} \rVert^2
\end{equation}

This is solved by 

\begin{equation}
\mathbf{P}_{\mathbf{\tilde{\Phi}} Y} = \left(\mathbf{\tilde{\Phi}}^T \mathbf{\tilde{\Phi}}+ \lambda \mathbf{I}\right)^{-1} \mathbf{\tilde{\Phi}}^T \mathbf{Y}
\end{equation}

or, by writing the last $\mathbf{\tilde{\Phi}}^T$ in terms of the kernel:

\begin{equation}
\mathbf{P}_{\mathbf{\tilde{\Phi}} Y} = \left(\mathbf{\tilde{\Phi}}^T \mathbf{\tilde{\Phi}}+ \lambda \mathbf{I}\right)^{-1} \mathbf{\Lambda}_{MM}^{-1/2} \mathbf{U}_{MM}^T \mathbf{K}_{NM}^T  \mathbf{Y}
\end{equation}

**Note**: For (kernel) ridge regression, we can use either the centered feature matrix $\mathbf{\tilde{\Phi}}$ or the uncentered feature matrix $\mathbf{\Phi}$. If we use $\mathbf{\tilde{\Phi}}$, the property matrix $\mathbf{Y}$ must also be centered for consistency.

In [ ]:
regularization = 1e-6

Let's start from after we've computed our sparse kernels and recompute $\mathbf{\tilde{\Phi}}$.

In [ ]:
%%time

vmm, Umm = np.linalg.eigh(Kmm)

# Umm/vmm are already sorted, but in *increasing* order, so reverse them
Umm = np.flip(Umm, axis=1)
vmm = np.flip(vmm, axis=0)

Umm = Umm[:, vmm > 0]
vmm = vmm[vmm > 0]

Phi = (
    Knm_train
    @ Umm[:, : n_active - 1]
    @ np.diagflat(1.0 / np.sqrt(vmm[0 : n_active - 1]))
)

PPY = Phi.T @ Phi
PPY = PPY + regularization * np.eye(Phi.shape[1])
PPY = np.linalg.pinv(PPY) @ Phi.T @ Y_train

## An Often Cheaper, More Elegant Route

We cast this expression into the more commonly used form by a series of simple manipulations that remove the need for diagonalizing $K_{MM}$ and computing $\mathbf{\tilde{\Phi}}$. First, we redefine the weights so that 

\begin{equation}
\mathbf{\tilde{\Phi}}\mathbf{P}_{\mathbf{\tilde{\Phi}} Y} = 
\mathbf{K}_{NM} \mathbf{U}_{MM} \mathbf{\Lambda}_{MM}^{-1/2} \mathbf{P}_{\mathbf{\tilde{\Phi}}Y} = 
\mathbf{K}_{NM} \tilde{\mathbf{P}_{K Y}}.
\end{equation}

Then,

\begin{align}
\tilde{\mathbf{P}_{K Y}}  &= 
\mathbf{U}_{MM} \mathbf{\Lambda}_{MM}^{-1/2}
\mathbf{P}_{\mathbf{\tilde{\Phi}} Y} \\
& = \mathbf{U}_{MM} \mathbf{\Lambda}_{MM}^{-1/2}
\left(\mathbf{\tilde{\Phi}}^T \mathbf{\tilde{\Phi}}+ \lambda \mathbf{I}_M\right)^{-1} 
\mathbf{\Lambda}_{MM}^{-1/2}  \mathbf{U}_{MM}^T 
\mathbf{K}_{NM}^T \mathbf{Y}\\
& = 
\left(\mathbf{U}_{MM}\mathbf{\Lambda}_{MM}^{1/2}\mathbf{\tilde{\Phi}}^T \mathbf{\tilde{\Phi}}\mathbf{\Lambda}_{MM}^{1/2}\mathbf{U}_{MM}^T+ \lambda \mathbf{U}_{MM}\mathbf{\Lambda}_{MM}\mathbf{U}_{MM}^T\right)^{-1} 
\mathbf{K}_{NM}^T \mathbf{Y}.
\end{align}

Now, by noting that 

\begin{equation}
\mathbf{U}_{MM} \mathbf{\Lambda}_{MM}^{1/2} 
\mathbf{\tilde{\Phi}}^T \mathbf{\tilde{\Phi}}
\mathbf{\Lambda}_{MM}^{1/2}  \mathbf{U}_{MM}^T  = 
\mathbf{K}_{NM}^T \mathbf{K}_{NM},
\end{equation}

we see that the sparse KRR model weights is computed by

\begin{equation}
\tilde{\mathbf{P}_{K Y}}  = 
\left(\mathbf{K}_{NM}^T \mathbf{K}_{NM}+ \lambda \mathbf{K}_{MM}\right)^{-1} 
\mathbf{K}_{NM}^T \mathbf{Y}.
\end{equation}


In [ ]:
%%time

PKY = Knm_train.T @ Knm_train + regularization * Kmm
PKY = np.linalg.pinv(PKY)
PKY = PKY @ Knm_train.T @ Y_train

As you see, this trick provides a (in some cases considerable) speed-up.

In [ ]:
Y_skrr_train = Knm_train @ PKY
Y_skrr_test = Knm_test @ PKY

We compare our results with those from KRR.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=dbl_fig)

ref_krr = KernelRidge(alpha=regularization, **kernel_params)
ref_krr.fit(X=K_train, y=Y_train)
Y_krr = ref_krr.predict(X=K_test)

plot_regression(Y_test[:, 0], Y_krr[:, 0], title="KRR", fig=fig, ax=axes[0], **cmaps)
plot_regression(
    Y_test[:, 0],
    Y_skrr_test[:, 0],
    title="Sparse KRR on {} Environments".format(n_active),
    fig=fig,
    ax=axes[1],
    **cmaps
)

## Error and Loss

Here our loss function is:

\begin{equation}
\ell_{regr} = \left\lVert \mathbf{Y} - \mathbf{K}_{NM}\tilde{\mathbf{P}_{K Y}}\right\rVert^2
\end{equation}

which we compare with KRR.

In [ ]:
table_from_dict(
    [
        get_stats(
            x=X_test,
            y=Y_test,
            yp=Y_krr,
        ),
        get_stats(
            x=X_test,
            y=Y_test,
            yp=Y_skrr_test,
        ),
    ],
    headers=["KRR", "sKRR"],
    title="Ridge Regression",
)

# Sparse KernelPCovR

For Sparse KernelPCovR, instead of using the Nystr&ouml;m approximation as in previous sparse methods, we formulate sparse KernelPCovR from KernelPCovR in a similar manner to how we derived feature space PCovR from sample space PCovR in the [PCovR Notebook](2_PrincipalCovariatesRegression.ipynb).

## A (Very) Quick Recap of Sample and Feature Space PCovR
In PCovR, we maximize the similarity

\begin{equation}
\rho = \operatorname{Tr}\left(\tilde{\mathbf{T}}^T\mathbf{\tilde{K}}\tilde{\mathbf{T}}\right),
\end{equation}

by taking as our whitened projection $\tilde{\mathbf{T}} = \mathbf{XP}_{X\tilde{T}}$ the eigenvectors corresponding to the $n_{PCA}$ largest eigenvalues of

\begin{equation}
    \mathbf{\tilde{K}} = \alpha {\mathbf{X} \mathbf{X}^T}
    + (1 - \alpha) {\hat{\mathbf{Y}} \hat{\mathbf{Y}}^T},
\end{equation}

which combines correlations between the samples in feature and property space. 

If the number of features is less than the number of samples, we can equivalently rewrite our similarity function as

\begin{align}
\rho &= \operatorname{Tr}\left(\mathbf{P}_{X\tilde{T}}^T\mathbf{C}^{1/2}\mathbf{C}^{-1/2}\mathbf{X}^T\mathbf{\tilde{K}}\mathbf{X}\mathbf{C}^{-1/2}\mathbf{C}^{1/2}\mathbf{P}_{X\tilde{T}}\right)
\end{align}

and diagonalize a modified covariance

\begin{equation}
\tilde{\mathbf{C}} = \mathbf{C}^{-1/2}\mathbf{X}^T\mathbf{\tilde{K}}\mathbf{X}\mathbf{C}^{-1/2}
\end{equation}

where $\mathbf{C} = \mathbf{X}^T\mathbf{X}$ to avoid diagonalizing the $n_{samples} \times n_{samples}$ matrix $\tilde{\mathbf{K}}$.

## Now we just do feature-space PCovR in the RKHS

In KernelPCovR, we maximize the similarity

\begin{equation}
\rho = \operatorname{Tr}\left(\tilde{\mathbf{T}}^T\mathbf{\tilde{K}}\tilde{\mathbf{T}}\right),
\end{equation}

however here $\tilde{\mathbf{T}} = \mathbf{KP}_{KT}$.
We compute the projection in feature space by maximizing:

\begin{align}
\rho = \operatorname{Tr}\left(\mathbf{P}_{K\tilde{T}}^T\mathbf{\tilde{\Phi}}\mathbf{\tilde{\Phi}}^T\mathbf{\tilde{K}}\mathbf{\tilde{\Phi}}\mathbf{\tilde{\Phi}}^T\mathbf{P}_{K\tilde{T}}\right)
\end{align}

where $\mathbf{K} = \mathbf{\tilde{\Phi}}\mathbf{\tilde{\Phi}}^T$.
It would make sense to use $\mathbf{\tilde{\Phi}}^T\mathbf{\tilde{K}}\mathbf{\tilde{\Phi}}$ as our sparse "kernel", but we must insert an identity to ensure that its eigenvectors are orthonormal. We use the covariance, defined as $\mathbf{C} = \mathbf{\tilde{\Phi}}^T\mathbf{\tilde{\Phi}}$, giving:

\begin{align}
\rho=\operatorname{Tr}\left(\mathbf{P}_{K\tilde{T}}^T\mathbf{\tilde{\Phi}}\mathbf{C}^{1/2}
\mathbf{C}^{-1/2}
\mathbf{\tilde{\Phi}}^T\mathbf{\tilde{K}}\mathbf{\tilde{\Phi}}\mathbf{C}^{-1/2}
\mathbf{C}^{1/2}
\mathbf{\tilde{\Phi}}^T\mathbf{P}_{K\tilde{T}}\right)
\end{align}

which ensures orthonormality, as 

\begin{align}
\left(\mathbf{P}_{K\tilde{T}}^T\mathbf{\tilde{\Phi}}\mathbf{C}^{1/2}
\mathbf{C}^{1/2}
\mathbf{\tilde{\Phi}}^T\mathbf{P}_{K\tilde{T}}\right)
&=\left(\mathbf{P}_{K\tilde{T}}^T\mathbf{\tilde{\Phi}}\mathbf{\tilde{\Phi}}^T\mathbf{\tilde{\Phi}}\mathbf{\tilde{\Phi}}^T\mathbf{P}_{K\tilde{T}}\right)\\
&=\left(\mathbf{P}_{K\tilde{T}}^T\mathbf{K}^T\mathbf{K}\mathbf{P}_{K\tilde{T}}\right)\\
&=\left(\tilde{\mathbf{T}}^T\tilde{\mathbf{T}}\right)\\
&=\mathbf{I}
\end{align}

In [ ]:
C = np.dot(Phi.T, Phi)

v_C, U_C = np.linalg.eigh(C)

# U_C/v_C are already sorted, but in *increasing* order, so reverse them
U_C = np.flip(U_C, axis=1)
v_C = np.flip(v_C, axis=0)

U_C = U_C[:, v_C > 0]
v_C = v_C[v_C > 0]


In analogy with to feature-space PCovR, we define

\begin{equation}
\mathbf{\tilde{C}} = \mathbf{C}^{-1/2}
\mathbf{\tilde{\Phi}}^T\mathbf{\tilde{K}}\mathbf{\tilde{\Phi}}\mathbf{C}^{-1/2}
\end{equation}

which evaluates to
\begin{equation}
\mathbf{\tilde{C}} = \alpha \frac{\mathbf{C}} {\operatorname{Tr}(\mathbf{C})/N} + (1 - \alpha) \mathbf{C}^{-1/2}\mathbf{\tilde{\Phi}}^T\mathbf{\hat{Y}}
\mathbf{\hat{Y}}^T\mathbf{\tilde{\Phi}}\mathbf{C}^{-1/2},
\end{equation}

Here we normalize our covariance matrix by the factor $\operatorname{Tr}(\mathbf{C})/N$ to achieve a balance between the KRR and KPCA parts.

**Note**: Since we use centered $\mathbf{\tilde{\Phi}}$ for the KPCA part, we have to use $\mathbf{\tilde{\Phi}}$ for the KRR part to keep it consistent. We can substitute $\mathbf{\hat{Y}}$ with the linear regression solution computed directly in active points RKHS: $\mathbf{Y}_{sKRR} = \mathbf{\tilde{\Phi}}\left(\mathbf{C}+ \lambda \mathbf{I}\right)^{-1}\mathbf{\tilde{\Phi}}^T \mathbf{Y}$.

<!---
\begin{equation}
\mathbf{\tilde{C}} = \alpha \frac{\mathbf{C}} {\operatorname{Tr}(\mathbf{C})/N} + (1 - \alpha) \mathbf{C}^{-1/2}
\mathbf{\Lambda}_{MM}^{-1/2}
\mathbf{U}_{MM}^T
\mathbf{K}_{NM}^T\mathbf{K}_{NM}
\left(\mathbf{K}_{NM}^T\mathbf{K}_{NM}+ \lambda \mathbf{K}_{MM}\right)^{-1}
\mathbf{K}_{NM}^T 
\mathbf{Y}
\mathbf{Y}^T
\mathbf{K}_{NM}
\left(\mathbf{K}_{NM}^T\mathbf{K}_{NM}+ \lambda \mathbf{K}_{MM}\right)^{-1}
\mathbf{K}_{NM}^T\mathbf{K}_{NM}
\mathbf{U}_{MM}
\mathbf{\Lambda}_{MM}^{-1/2}
\mathbf{C}^{-1/2}
\end{equation}

\begin{equation}
\mathbf{\tilde{C}} = \alpha \frac{\mathbf{C}} {\operatorname{Tr}(\mathbf{C})/N} + (1 - \alpha) \mathbf{C}^{-1/2}
\mathbf{\Lambda}_{MM}^{-1/2}
\mathbf{U}_{MM}^T
\left(\mathbf{I}_{M}+ \lambda 
\mathbf{K}_{MM}\left(\mathbf{K}_{NM}^T\mathbf{K}_{NM}\right)^{-1}
\right)^{-1}
\mathbf{K}_{NM}^T 
\mathbf{Y}
\mathbf{Y}^T
\mathbf{K}_{NM}
\left(\mathbf{I}_{M}+ \lambda 
\mathbf{K}_{MM}\left(\mathbf{K}_{NM}^T\mathbf{K}_{NM}\right)^{-1}
\right)^{-1}
\mathbf{U}_{MM}
\mathbf{\Lambda}_{MM}^{-1/2}
\mathbf{C}^{-1/2}
\end{equation}
--->

In [ ]:
alpha = 0.5
regularization = 1e-6

Csqrt = U_C @ np.diagflat(np.sqrt(v_C)) @ U_C.T
iCsqrt = U_C @ np.diagflat(1.0 / np.sqrt(v_C)) @ U_C.T

C_pca = C / (np.trace(C) / C.shape[0])

C_lr = np.linalg.pinv(C + regularization * np.eye(C.shape[0]))
C_lr = iCsqrt @ Phi.T @ Phi @ C_lr @ Phi.T @ Y_train.reshape(-1, Y_train.shape[-1])
C_lr = C_lr @ C_lr.T

Ct = alpha * C_pca + (1 - alpha) * C_lr

We can then find the eigendecomposition of 
$\mathbf{\tilde{C}}=
\mathbf{U}_\mathbf{\tilde{C}}\mathbf{\Lambda}_\mathbf{\tilde{C}}\mathbf{U}_\mathbf{\tilde{C}}^T$  and 
solve for $\mathbf{P}_{\tilde{\Phi} T}$ (again analogous to feature-space PCovR, swapping $\mathbf{\tilde{\Phi}}$ for $\mathbf{X}$): 

\begin{equation}
\mathbf{P}_{\tilde{\Phi} T}=\mathbf{C}^{-1/2}\mathbf{\hat{U}}_\mathbf{\tilde{C}}\mathbf{\hat{\Lambda}}_\mathbf{\tilde{C}}^{1/2} 
\end{equation}

where the $\hat{\cdot}$ decoration denotes a truncation to $n_{PCA}$ components, as usual.

In [ ]:
v_Ct, U_Ct = np.linalg.eigh(Ct)

# U_Ct/v_Ct are already sorted, but in *increasing* order, so reverse them
U_Ct = np.flip(U_Ct, axis=1)
v_Ct = np.flip(v_Ct, axis=0)

U_Ct = U_Ct[:, v_Ct > 0]
v_Ct = v_Ct[v_Ct > 0]

PPT = iCsqrt @ U_Ct[:, :n_PC] @ np.diag(np.sqrt(v_Ct[:n_PC]))
v_Ct.shape

## Projecting into Latent Space

Our projection in feature space takes the form:

\begin{equation}
\mathbf{T} = \mathbf{\tilde{\Phi}}_{NM}\mathbf{P}_{\tilde{\Phi} T}
\end{equation}

If we want to project using a kernel rather than a feature space vector, this becomes:

\begin{align}
\mathbf{T} &=  \left(\mathbf{K}_{NM}-\mathbf{\bar{K}}_M\right)\mathbf{U}_{MM}\mathbf{\Lambda}_{MM}^{-1/2}\mathbf{P}_{\tilde{\Phi} T} \\
&=\mathbf{K}_{NM}\mathbf{U}_{MM}\mathbf{\Lambda}_{MM}^{-1/2}\mathbf{C}^{-1/2}\mathbf{\hat{U}}_\mathbf{\tilde{C}}\mathbf{\hat{\Lambda}}_\mathbf{\tilde{C}}^{1/2} - \mathbf{\bar{\Phi}}\mathbf{P}_{\tilde{\Phi} T} \\
&= \mathbf{K}_{NM}\mathbf{P}_{KT} -  \mathbf{\bar{T}}
\end{align}

where $\mathbf{P}_{K T} = \mathbf{P}_{K\Phi} \mathbf{P}_{\Phi T} = 
\mathbf{U}_{MM}\mathbf{\Lambda}_{MM}^{-1/2}\mathbf{C}^{-1/2}\mathbf{\hat{U}}_\mathbf{\tilde{C}}\mathbf{\hat{\Lambda}}_\mathbf{\tilde{C}}^{1/2} $.

In [ ]:
PKT = Umm[:, :n_active-1] @ np.diagflat(1/np.sqrt(vmm[:n_active-1])) @ PPT

In [ ]:
T =  Knm_train @ PKT

In [ ]:
T_skpcovr_test = Knm_test @ PKT

We again compare to the non-sparse kernel version, giving

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=dbl_fig)

ref = KernelPCovR(mixing=alpha, n_components=2, center=True, **kernel_params)
ref.fit(X_train, Y_train)
t = ref.transform(X_test)
y = ref.predict(X_test)

plot_projection(
    Y_test,
    check_mirrors(T_skpcovr_test, t),
    fig=fig,
    ax=axes[0],
    title="Sparse KernelPCovR",
    **cmaps
)
plot_projection(Y_test, t, fig=fig, ax=axes[1], title="KernelPCovR", **cmaps)

plt.show()

## Predicting the Properties

Property prediction takes the exact same form as in KernelPCovR, except with $\mathbf{T}$ supplied by our sparse construction.

In [ ]:
PTY = T.T @ T
PTY = np.linalg.pinv(PTY) @ T.T @ Y_train

In [ ]:
Ypred = Knm_test @ PKT @ PTY

fig, axes = plt.subplots(1, 2, figsize=dbl_fig)

plot_regression(
    Y_test[:, 0],
    Ypred[:, 0],
    fig=fig,
    ax=axes[0],
    title=f"Sparse KernelPCovR with {n_active} Environments",
    **cmaps,
)
plot_regression(Y_test[:, 0], y[:, 0], fig=fig, ax=axes[1], title="KernelPCovR", **cmaps)

plt.show()

# Next: CUR Decomposition and Feature Selection

Continue on to the [next notebook](5_CUR.ipynb).

# Implementation in `skcosmo`

Here we have directly implemented sparse methods, many of which do not require the explicit determination of the feature vectors $\mathbf{\Phi}$. Instead, it is possible to determine the feature vectors and use simple, linear models for the regression and dimensionality reduction.

Currently, `scikit-learn` gives a feature generator under kernel_approximation.Nystroem. This determines a sparse kernel with a random active set, and in `scikit-cosmo` we are actively developing a version where the active set can be predetermined.

In [ ]:
from sklearn.kernel_approximation import Nystroem